https://python.langchain.com/docs/use_cases/sql/quickstart/

### **Test the sqldb**

In [1]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

**Connecting to the sqldb**

In [2]:
db_path = str(here("data")) + "/sqldb.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [3]:
db

In [4]:
# validate the connection to the vectordb
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

### **Test the access to the environment variables**

In [5]:
from dotenv import load_dotenv
import os
print("Environment variables are loaded:", load_dotenv())
print("test by reading a variable:", os.getenv("gpt_deployment_name"))

Environment variables are loaded: True
test by reading a variable: gemini-2.5-flash


### **Test your GPT model**

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage

client = ChatGoogleGenerativeAI(
            model=os.getenv("gpt_deployment_name"),
            temperature=0.7,
            google_api_key=os.getenv("GOOGLE_API_KEY"),
        )

system_msg = SystemMessage(
            content="You are a helpful assistant"
        )
user_prompt = "hello"
user_msg = HumanMessage(content=user_prompt)
response = client.invoke([system_msg, user_msg])
response.content

'Hello! How can I help you today?'

### **1. SQL query chain**

In [ ]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(client, db)

import re

def extract_sql_query(text: str) -> str:
    """
    Extract a clean SQL query from a possibly formatted or prefixed LLM output.
    Handles code blocks, 'SQLQuery:' prefix, and various SQL command types.
    """
    text = text.strip()

    # Step 1: Remove code block markers
    if text.startswith("```"):
        lines = text.splitlines()
        # Remove first/last lines if they're ``` or ```sql
        if lines[0].startswith("```") and lines[-1].startswith("```"):
            text = "\n".join(lines[1:-1]).strip()

    # Step 2: Remove known prefixes like "SQLQuery:"
    text = re.sub(r"^(SQLQuery:|Query:)\s*", "", text, flags=re.IGNORECASE)

    # Step 3: Extract only the actual SQL statement (any type)
    match = re.search(
        r"(SELECT|INSERT|UPDATE|DELETE|PRAGMA|CREATE|DROP|ALTER|DESCRIBE|SHOW)\s.+",
        text,
        flags=re.IGNORECASE | re.DOTALL,
    )
    if match:
        query = match.group(0).strip()
        # Remove trailing backticks or semicolons
        return query.rstrip(";`").strip()

    # Step 4: If nothing matched, return original (but cleaned)
    return text


response = chain.invoke({"question": "How many employees are there"})
sql_query = extract_sql_query(response)
print(sql_query)

-- This question cannot be answered by a SQL query.
-- The "most important data" is subjective and depends on the context or purpose of the database.
-- In the PlaylistTrack table, both "PlaylistId" and "TrackId" are equally important as they together form the primary key, linking playlists to tracks.


Execute the query to make sure it’s valid

In [8]:
db.run(sql_query)

''

In [ ]:
chain.get_prompts()[0].pretty_print()

### **Add QuerySQLDataBaseTool to the chain**
Execute SQL query

**This is the most dangerous part of creating a SQL chain.** Consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to you chains before query execution (see below).

We can use the QuerySQLDatabaseTool to easily add query execution to our chain:

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.runnables import RunnableLambda

write_query = create_sql_query_chain(client, db)
execute_query = QuerySQLDataBaseTool(db=db)

clean_query_output = RunnableLambda(extract_sql_query)

chain = write_query | clean_query_output | execute_query

chain.invoke({"question": "How many employees are there"})


### **Answer the question in a user friendly manner**

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | client | StrOutputParser()
chain = (
    RunnablePassthrough.assign(raw_query=write_query)
    .assign(query=itemgetter("raw_query") | clean_query_output)
    .assign(result=itemgetter("query") | execute_query)
    | answer
)

chain.invoke({"question": "How many employees are there"})

### **2. Agents**

Agent which provides a more flexible way of interacting with SQL databases. The main advantages of using the SQL Agent are:

- It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
- It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
- It can answer questions that require multiple dependent queries.
- It will save tokens by only considering the schema from relevant tables.

To initialize the agent, we use create_sql_agent function. This agent contains the SQLDatabaseToolkit which contains tools to:

- Create and execute queries
- Check query syntax
- Retrieve table descriptions
- …

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(client, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)

In [ ]:
agent_executor.invoke({"input": "Describe the playlisttrack table"})
# agent_executor.invoke("Describe the playlisttrack table")